# Análise Comparativa


Essa etapa tem como objetivo comparar alguns modelos para encontrar um que possa ser melhor utilizado dentro do problema, para isso antes realizamos a preparação e o pré-processamento dos dados.

## Preparação dos dados

In [ ]:
from IPython.display import Markdown
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, ShuffleSplit, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import mean_squared_error

In [ ]:
# Ler o csv
df_day = pd.read_csv('day.csv')

# Ler o dicionário de dados
df_dict = pd.read_csv('dicionario_dados_day.csv')
df_day.head(1)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985


### Tratamento dos dados

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados. Através do dicionário de dados apresentado no notebook/01_exploratory_data_analysis, é possível perceber que as colunas *casual* e *registered* são deviradas da nossa variável alvo, ou seja, a soma das duas é equivalente ao resultado da coluna **cnt**. Portanto, serão excluídos durante o tratamento e não seguirão para o experimento. A coluna *dteday* também não possui informações relevantes para esta análise.

Foi verificado anteriormente que o dataset não possui valores nulos ou outliers.

Além disso, as colunas categóricas já estão codificadas em sequencias numéricas, enquanto as variáveis contínuas estão calculadas dentro da mesma escala.

In [ ]:
df_day = df_day.drop(['casual', 'registered', 'dteday', 'instant'], axis = 1)
df_day.head(2)

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801


In [ ]:
df_day.columns

Index(['instant', 'season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday',
       'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'cnt'],
      dtype='object')

- Tentativa


In [ ]:
target_column = 'cnt' #Variável alvo

#Obter uma lista das variáveis discretas
discrete_columns = (
    df_dict
    .query('tipo == "Discreta" and variavel != @target_column')
    .variavel
    .to_list()
)

#Obter uma lista das váriaveis nominais
nominal_columns = (
    df_dict
    .query('tipo == "Nominal"')
    .variavel
    .to_list()
)

#Obter uma lista das váriaveis contínuas
continuous_columns = (
    df_dict
    .query('tipo == "Contínua"')
    .variavel
    .to_list()
)

#Obter uma lista das váriaveis ordinais
ordinal_columns = (
    df_dict
    .query('tipo == "Ordinal"')
    .variavel
    .to_list()
)

In [ ]:
#Para cada tipo de variavel, será criado uma pipeline para tratar

discrete_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='mean')), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', MinMaxScaler()) # Normalização
])

nominal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoder', OneHotEncoder(sparse=False)), # Codificação de variáveis
    # Seleção de variáveis
    ('normalization', MinMaxScaler()), # Normalização
]),
continuous_preprocessor = Pipeline(steps=[
    # Tratamento de dados discrepantes
    ('missing', KNNImputer(n_neighbors=5)), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', RobustScaler()) # Normalização
]),
ordinal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoder', OneHotEncoder(sparse=False)), # Codificação de variáveis
    # Seleção de variáveis
    ('normalization', MinMaxScaler())]) # Normalização

In [ ]:
# Transformando os dados utilizando os pré-processadores criados acima
preprocessing = ColumnTransformer([
    ("discrete", discrete_preprocessor, discrete_columns),
    ("nominal", nominal_preprocessor, nominal_columns),
    ("continuous", continuous_preprocessor, continuous_columns),
    ("ordinal", ordinal_preprocessor, ordinal_columns)
])
preprocessing

ColumnTransformer(transformers=[('discrete',
                                 Pipeline(steps=[('missing', SimpleImputer()),
                                                 ('normalization',
                                                  MinMaxScaler())]),
                                 []),
                                ('nominal',
                                 (Pipeline(steps=[('missing',
                                                   SimpleImputer(strategy='most_frequent')),
                                                  ('encoder',
                                                   OneHotEncoder(sparse=False)),
                                                  ('normalization',
                                                   MinMaxScaler())]),),
                                 []),
                                ('continuous',
                                 (Pipeline(steps=[('missing', KNNImputer()),
                                                  ('normalization',
                                                   RobustScaler())]),),
                                 []),
                                ('ordinal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(sparse=False)),
                                                 ('normalization',
                                                  MinMaxScaler())]),
                                 [])])

## Metodologia

Será utilizado o método de validação cruzada **k-fold**. Além disso, serão apresentados os seguintes modelos:

- Regressão Linear

- Randon forest

- Gradiente


## Configuração do experimento

- Separar o conjunto em treino e teste

In [ ]:
X = df_day.drop(columns=[target_column], axis=1)
y = (
    df_day[[target_column]]
    .replace({"YES": 1, "NO": 0})
    .to_numpy()
    .ravel()
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

- Realizar teste com os modelos definidos

In [ ]:
# validação cruzada

model = LinearRegression()

# validação cruzada


model.fit(X_train, y_train)
y_test_hat = model.predict(X_test)

mse = mean_squared_error(y_test, y_test_hat)
print(mse)

665798.4968239996


## Resultados e discussão

- Expor em uma tabela a comparação dos modelos para explicar qual deles melhor se encaixa para o projeto